In [1]:
import ast
import dis
import astpretty

# Parsing: ast module

In [2]:
code = "one_plus_two=1+2"
code = "print('hello there.')"
code1 = "a=1"
code2 = "b=1"

code6 = """    
x = 80
a = 100
g = 238

# Multiplies
def mul(x, y):
    return x*y

print(g + a)
z = mul(x, k)
"""

In [3]:
code_11 = """
c = 7
b = 7
if a > 7 :
    print(a)
"""

In [7]:
tree0 = ast.parse(code_11, mode="exec")
b0 = compile(tree0, "<string>", mode="exec")
print(b0.co_code)

b'\x97\x00d\x00Z\x00d\x00Z\x01e\x02d\x00k\x04\x00\x00\x00\x00r\r\x02\x00e\x03e\x02\xa6\x01\x00\x00\xab\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00d\x01S\x00d\x01S\x00'


In [29]:
# Get AST 
tree = ast.parse(code6, mode="exec") # k is undefined, but it didn't catch the error
print(ast.dump(tree, indent=2))
# astpretty.pprint(tree)

Module(
  body=[
    Assign(
      targets=[
        Name(id='a', ctx=Store())],
      value=Constant(value=100)),
    Assign(
      targets=[
        Name(id='b', ctx=Store())],
      value=Constant(value=238)),
    Expr(
      value=Call(
        func=Name(id='print', ctx=Load()),
        args=[
          BinOp(
            left=Name(id='b', ctx=Load()),
            op=Add(),
            right=Name(id='a', ctx=Load()))],
        keywords=[]))],
  type_ignores=[])


In [5]:
# Idea! Make all variables and const with the same value

code06 = """    
a = 1
a = 1
a = 1

# Multiplies
def mul(a, a):
    return a*a

print(a + a)
z = mul(a, a)
"""

In [6]:
print(ast.dump(ast.parse(code06), indent=2))

Module(
  body=[
    Assign(
      targets=[
        Name(id='a', ctx=Store())],
      value=Constant(value=1)),
    Assign(
      targets=[
        Name(id='a', ctx=Store())],
      value=Constant(value=1)),
    Assign(
      targets=[
        Name(id='a', ctx=Store())],
      value=Constant(value=1)),
    FunctionDef(
      name='mul',
      args=arguments(
        posonlyargs=[],
        args=[
          arg(arg='a'),
          arg(arg='a')],
        kwonlyargs=[],
        kw_defaults=[],
        defaults=[]),
      body=[
        Return(
          value=BinOp(
            left=Name(id='a', ctx=Load()),
            op=Mult(),
            right=Name(id='a', ctx=Load())))],
      decorator_list=[]),
    Expr(
      value=Call(
        func=Name(id='print', ctx=Load()),
        args=[
          BinOp(
            left=Name(id='a', ctx=Load()),
            op=Add(),
            right=Name(id='a', ctx=Load()))],
        keywords=[])),
    Assign(
      targets=[
        Name(id='z', ctx=

In [7]:
# From AST back to src code
print(ast.unparse(tree))

x = 80
a = 100
g = 238

def mul(x, y):
    return x * y
print(g + a)
z = mul(x, k)


In [8]:
# Traverse AST 
for i, node in enumerate(ast.walk(tree)):
    print(type(node).__name__)

    if i == 4:
        break

Module
Assign
Assign
Assign
FunctionDef


In [9]:
# Modifing tree

class ReplaceAdd(ast.NodeTransformer):
    def visit_BinOp(self, node):
        self.generic_visit(node)
        if (isinstance(node.op, ast.Add) and 
            isinstance(node.left, ast.Constant) and node.left.value == 2 and 
            isinstance(node.right, ast.Constant) and node.right.value == 3):
            return ast.Constant(value=99)
        return node

code = "x = 2 + 3"
tree_exp = ast.parse(code)
new_tree = ReplaceAdd().visit(tree_exp)
new_code = ast.unparse(new_tree)
print(new_code)  #

x = 99


### Types of errors
- Parsing time
    - Missing punctuation (e.g., :, ), ], }).
    - Incorrect indentation.
    - Invalid use of keywords or operators.
    - Incomplete statements.
- Compilation time
    - due to invalid ASTs (e.g., after manual modification) or incorrect arguments.
- Execution time
    - logical errors
    - NameError, ...

In [10]:
# Only execution time error 
exec(compile(ast.parse("print(v)"), '<string>', mode="exec"))

NameError: name 'v' is not defined

# Compilation

In [11]:
code0 = "a=1\nb=2"
tree0 = ast.parse(code0, mode="exec")
print("AST")
print(ast.dump(tree0, indent=4))
print()

b0 = compile(tree0, "<string>", mode="exec")
print("Bytecode")
print([b for b in b0.co_code])

AST
Module(
    body=[
        Assign(
            targets=[
                Name(id='a', ctx=Store())],
            value=Constant(value=1)),
        Assign(
            targets=[
                Name(id='b', ctx=Store())],
            value=Constant(value=2))],
    type_ignores=[])

Bytecode
[151, 0, 100, 0, 90, 0, 100, 1, 90, 1, 100, 2, 83, 0]


In [12]:
# Get bytecode object
# The compile() function in Python converts source code or an Abstract Syntax Tree (AST) 
# into a code object that you can execute with exec() or evaluate with eval().
compiled = compile(tree0, "<string>", mode="exec")
type(compiled)

code

In [13]:
# Show bytecode
dis.dis(compiled)

  0           0 RESUME                   0

  1           2 LOAD_CONST               0 (1)
              4 STORE_NAME               0 (a)

  2           6 LOAD_CONST               1 (2)
              8 STORE_NAME               1 (b)
             10 LOAD_CONST               2 (None)
             12 RETURN_VALUE


In [14]:
def are_equal(l1, l2):
    for ll1, ll2 in zip(l1.co_code, l2.co_code):
        if ll1 != ll2:
            print(False)
            return

    print(True)  

### Example 1: Variable names, values and const folding don't matter

In [15]:
code1 = "a=1"
b1 = compile(ast.parse(code1, mode="exec"), "<string>", mode="exec")

In [16]:
code2 = "b=7"
b2 = compile(ast.parse(code2, mode="exec"), "<string>", mode="exec")

In [17]:
code3 = "c=7+9"
b3 = compile(ast.parse(code3, mode="exec"), "<string>", mode="exec")

In [18]:
code4 = "c=a+9"
b4 = compile(ast.parse(code4, mode="exec"), "<string>", mode="exec")

In [19]:
are_equal(b1, b2)
are_equal(b2, b3)
are_equal(b2, b4)

True
True
False


### Example 2: Switching places of variables do matter

In [7]:
code5 = """
a = 100
b = 238
print(a + b)
"""
b5 = compile(ast.parse(code5, mode="exec"), "<string>", mode="exec")

In [8]:
code6 = """
a = 100
b = 238
print(b + a)
"""
b6 = compile(ast.parse(code6, mode="exec"), "<string>", mode="exec")

In [22]:
code7 = """
a = 100
b = 238
print(a + b + 100)
"""
b7 = compile(ast.parse(code7, mode="exec"), "<string>", mode="exec")

In [23]:
code8 = """
a = 100
b = 238
print(a + b + 100 + 50)
"""
b8 = compile(ast.parse(code8, mode="exec"), "<string>", mode="exec")

In [24]:
are_equal(b5, b6)
are_equal(b5, b7)
are_equal(b7, b8)

False
False
False


In [25]:
print(ast.dump(ast.parse(code7), indent=2))

Module(
  body=[
    Assign(
      targets=[
        Name(id='a', ctx=Store())],
      value=Constant(value=100)),
    Assign(
      targets=[
        Name(id='b', ctx=Store())],
      value=Constant(value=238)),
    Expr(
      value=Call(
        func=Name(id='print', ctx=Load()),
        args=[
          BinOp(
            left=BinOp(
              left=Name(id='a', ctx=Load()),
              op=Add(),
              right=Name(id='b', ctx=Load())),
            op=Add(),
            right=Constant(value=100))],
        keywords=[]))],
  type_ignores=[])


In [26]:
print(ast.dump(ast.parse(code8), indent=2))

Module(
  body=[
    Assign(
      targets=[
        Name(id='a', ctx=Store())],
      value=Constant(value=100)),
    Assign(
      targets=[
        Name(id='b', ctx=Store())],
      value=Constant(value=238)),
    Expr(
      value=Call(
        func=Name(id='print', ctx=Load()),
        args=[
          BinOp(
            left=BinOp(
              left=BinOp(
                left=Name(id='a', ctx=Load()),
                op=Add(),
                right=Name(id='b', ctx=Load())),
              op=Add(),
              right=Constant(value=100)),
            op=Add(),
            right=Constant(value=50))],
        keywords=[]))],
  type_ignores=[])


In [27]:
code9 = """
a = 80
a = 100
a = 238
print(a + a)
"""

b9 = compile(ast.parse(code9, mode="exec"), "<string>", mode="exec")

code10 = """
b = 80
b = 100+90
b = 238
print(b + b)
"""

b10 = compile(ast.parse(code10, mode="exec"), "<string>", mode="exec")

are_equal(b9, b10)

True


In [28]:
code9 = """
a = 80
b = a + 5
print(b)
"""

b9 = compile(ast.parse(code9, mode="exec"), "<string>", mode="exec")

code10 = """
a = 80
b = a + 10 
print(b)
"""

b10 = compile(ast.parse(code10, mode="exec"), "<string>", mode="exec")

are_equal(b9, b10)

True


In [37]:
code_1 = "x = 5"
code_2 = "z = 2 + 3"

In [41]:
tree = ast.parse(code_1, mode="exec")
print(ast.dump(tree, indent=2))

Module(
  body=[
    Assign(
      targets=[
        Name(id='x', ctx=Store())],
      value=Constant(value=5))],
  type_ignores=[])


In [42]:
tree = ast.parse(code_2, mode="exec") 
print(ast.dump(tree, indent=2))

Module(
  body=[
    Assign(
      targets=[
        Name(id='z', ctx=Store())],
      value=BinOp(
        left=Constant(value=2),
        op=Add(),
        right=Constant(value=3)))],
  type_ignores=[])


In [43]:
are_equal(compile(ast.parse(code_1, mode="exec"), "<string>", mode="exec"), compile(ast.parse(code_2, mode="exec"), "<string>", mode="exec"))

True


In [44]:
code_1 = "a=2\nb=3"
code_2 = "c=4\nd=5"

In [45]:
are_equal(compile(ast.parse(code_1, mode="exec"), "<string>", mode="exec"), compile(ast.parse(code_2, mode="exec"), "<string>", mode="exec"))

True


In [53]:
code_1 = "a=2\nb=3\nif a==b: \n    print('yes')\nelse:\n    print('no')"
code_2 = "c=4\nd=5\nif c==d: \n    print('yes')\nelse:\n    print('no')"
code_3 = "c=4\nd=5\nif c!=d: \n    print('no')\nelse:\n    print('yes')"

In [54]:
are_equal(compile(ast.parse(code_2, mode="exec"), "<string>", mode="exec"), compile(ast.parse(code_3, mode="exec"), "<string>", mode="exec"))

False


In [3]:
class CanonicalizeCommutativeOps(ast.NodeTransformer):
    def visit_BinOp(self, node):
        self.generic_visit(node)

        if isinstance(node.op, (ast.Add, ast.Mult)):
            operands = self.flatten_commutative_chain(node, type(node.op))
            sorted_operands = sorted(operands, key=self.sort_key)

            return self.build_commutative_chain(sorted_operands, node.op)

        return node

    def flatten_commutative_chain(self, node, op_type):
        if isinstance(node, ast.BinOp) and isinstance(node.op, op_type):

            return self.flatten_commutative_chain(node.left, op_type) + \
                   self.flatten_commutative_chain(node.right, op_type)
        else:
            return [node]

    def build_commutative_chain(self, operands, op):
        if not operands:
            return ast.Constant(value=0 if isinstance(op, ast.Add) else 1)

        expr = operands[0]

        for operand in operands[1:]:
            expr = ast.BinOp(left=expr, op=op, right=operand)

        return expr

    def sort_key(self, node):
        if isinstance(node, ast.Constant):
            return (0, repr(node.value))
        elif isinstance(node, ast.Name):
            return (1, node.id)
        else:
            return (2, ast.dump(node))


def normalize_code(code):
    tree = ast.parse(code)
    normalized = CanonicalizeCommutativeOps().visit(tree)
    ast.fix_missing_locations(normalized)

    return ast.unparse(normalized)

In [10]:

code = "a = (b + 8) - (a + 3)\nb = b + a + 1"
print(normalize_code(code))

a = 8 + b - (3 + a)
b = 1 + a + b


In [9]:
code_5_norm = normalize_code(code5)
code_6_norm = normalize_code(code6)

In [10]:
code_5_norm

'a = 100\nb = 238\nprint(a + b)'

In [11]:
code_6_norm

'a = 100\nb = 238\nprint(a + b)'

In [1]:
import ast
import builtins

def find_undeclared_vars(source):
    tree = ast.parse(source)
    assigned = set()
    used = set()

    for node in ast.walk(tree):
        # any assignment target
        if isinstance(node, ast.Name) and isinstance(node.ctx, ast.Store):
            assigned.add(node.id)
        # any usage
        if isinstance(node, ast.Name) and isinstance(node.ctx, ast.Load):
            used.add(node.id)

    # drop builtins (print, len, etc.)
    builtins_names = set(dir(builtins))
    undeclared = used - assigned - builtins_names
    return undeclared

In [2]:
code1 = """
c = 7
b = 7
d = (a / 7) + (a * 7)
"""
print(find_undeclared_vars(code1))

{'a'}


In [4]:
code2 = """
a = 8
if a > 7:
    print(a)
elif b <= b:
    print(b)
else:
    print(c)
"""
print(find_undeclared_vars(code2))  

{'b', 'c'}
